### Imports

In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import scipy.signal
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure
import random

Using TensorFlow backend.


### Data Load

In [78]:
# data = pandas.read_csv('./data/Learning/brake, acc/Train_Set.csv')
# test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')
data = pandas.read_csv('./data/Learning/MyData/version1/Train_Set.csv')
test = pandas.read_csv('./data/Learning/MyData/version1/Test_Set.csv')
# data = pandas.read_csv('./data/Learning/MyData/version2/Train_Set.csv')
# test = pandas.read_csv('./data/Learning/MyData/version2/Test_Set.csv')

# data = data[0:106092]
# data = data[0:105658]
x_train_set = data[['x','y','z']]
# x_train_set = data[['x']]

target = data['AccStatus']

x_test_set = test[['x','y','z']]
# x_test_set = test[['x']]
y_test_set = test['AccStatus']


x_train_set = np.array(x_train_set,dtype=float)          
target = np.array(target,dtype=float)

x_test_set =  np.array(x_test_set,dtype=float)
y_test_set =  np.array(y_test_set,dtype=float)

In [79]:
len(x_train_set)

106105

In [48]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [80]:
x_train_set = SmoothData(x_train_set)
x_test_set = SmoothData(x_test_set)

### One-hot encoding

In [81]:
target_onehot  = []
y_test_set_onehot = []
for i in range(len(target)):
    if(target[i]==0):
        target_onehot.append(np.array([1,0,0]))
    elif (target[i]==1):
        target_onehot.append(np.array([0,1,0]))
    elif(target[i]==2):
        target_onehot.append(np.array([0,0,1]))
        
for i in range(len(y_test_set)):
    if(y_test_set[i]==0):
        y_test_set_onehot.append(np.array([1,0,0]))
    elif (y_test_set[i]==1):
        y_test_set_onehot.append(np.array([0,1,0]))
    elif (y_test_set[i]==2):
        y_test_set_onehot.append(np.array([0,0,1]))

### LSTM one-hot

In [90]:
shuffle_x_train = []
shuffle_target = []
shuffle_index = random.sample(range(0, 104000), 1040)
for i in shuffle_index:
    shuffle_x_train.append (np.array([x_train_set[i:i+200]]).reshape(200,3))
    shuffle_target.append (np.array([target_onehot[i:i+200]]).reshape(200,3))

In [57]:
x_train_set[111900:112900]

array([[-0.02476153, -0.06021106, -0.00557274],
       [-0.00297171, -0.05836046, -0.00724584],
       [-0.02217192, -0.06739837, -0.04718351],
       ...,
       [-0.00102573, -0.08944807,  0.03003007],
       [-0.01715303, -0.09017074,  0.03510501],
       [-0.01485797, -0.08390096,  0.03308372]])

In [85]:
len(x_test_set)

34427

In [91]:
model = Sequential()
model.add(LSTM(10, input_shape = (200,3), return_sequences = True))
model.add(LSTM(10, return_sequences= True))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 200, 10)           560       
_________________________________________________________________
lstm_12 (LSTM)               (None, 200, 10)           840       
_________________________________________________________________
dense_6 (Dense)              (None, 200, 3)            33        
Total params: 1,433
Trainable params: 1,433
Non-trainable params: 0
_________________________________________________________________


In [84]:
history = model.fit( 
    np.array([x_train_set[i:i+200] for i in range(0,104000,100)]),
    np.array([target_onehot[i:i+200] for i in range(0,104000,100)]),
    epochs=200,
    validation_data = (np.array([x_train_set[i:i+200] for i in range(104000,105000,50)]),
                     np.array([target_onehot[i:i+200] for i in range(104000,105000,50)])),
    batch_size=64)

Train on 1040 samples, validate on 20 samples
Epoch 1/200
1040/1040 [==============================] - 8s 8ms/step - loss: 0.9931 - categorical_accuracy: 0.8731 - val_loss: 0.7982 - val_categorical_accuracy: 0.8890
Epoch 2/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.5838 - categorical_accuracy: 0.9020 - val_loss: 0.5130 - val_categorical_accuracy: 0.8890
Epoch 3/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4454 - categorical_accuracy: 0.9020 - val_loss: 0.4865 - val_categorical_accuracy: 0.8890
Epoch 4/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4238 - categorical_accuracy: 0.9020 - val_loss: 0.4529 - val_categorical_accuracy: 0.8890
Epoch 5/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4064 - categorical_accuracy: 0.9020 - val_loss: 0.4342 - val_categorical_accuracy: 0.8890
Epoch 6/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.3918 - categorical_accuracy: 0

Epoch 49/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2728 - categorical_accuracy: 0.9106 - val_loss: 0.4706 - val_categorical_accuracy: 0.9105
Epoch 50/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2795 - categorical_accuracy: 0.9111 - val_loss: 0.4215 - val_categorical_accuracy: 0.9105
Epoch 51/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2779 - categorical_accuracy: 0.9104 - val_loss: 0.3433 - val_categorical_accuracy: 0.9107
Epoch 52/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2750 - categorical_accuracy: 0.9111 - val_loss: 0.3881 - val_categorical_accuracy: 0.9115
Epoch 53/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2742 - categorical_accuracy: 0.9105 - val_loss: 0.3949 - val_categorical_accuracy: 0.9112
Epoch 54/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2773 - categorical_accuracy: 0.9113 - val_loss: 0.3984 - val_categoric

1040/1040 [==============================] - 6s 6ms/step - loss: 0.2546 - categorical_accuracy: 0.9121 - val_loss: 0.3667 - val_categorical_accuracy: 0.9140
Epoch 98/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2597 - categorical_accuracy: 0.9118 - val_loss: 0.4306 - val_categorical_accuracy: 0.9130
Epoch 99/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2538 - categorical_accuracy: 0.9117 - val_loss: 0.4118 - val_categorical_accuracy: 0.9137
Epoch 100/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2513 - categorical_accuracy: 0.9114 - val_loss: 0.3586 - val_categorical_accuracy: 0.9145
Epoch 101/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2516 - categorical_accuracy: 0.9115 - val_loss: 0.3610 - val_categorical_accuracy: 0.9145
Epoch 102/200
1040/1040 [==============================] - 6s 5ms/step - loss: 0.2506 - categorical_accuracy: 0.9113 - val_loss: 0.4875 - val_categorical_accurac

1040/1040 [==============================] - 4s 4ms/step - loss: 0.2392 - categorical_accuracy: 0.9159 - val_loss: 0.4151 - val_categorical_accuracy: 0.9183
Epoch 146/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2400 - categorical_accuracy: 0.9169 - val_loss: 0.3330 - val_categorical_accuracy: 0.9180
Epoch 147/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2427 - categorical_accuracy: 0.9159 - val_loss: 0.3807 - val_categorical_accuracy: 0.9178
Epoch 148/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2385 - categorical_accuracy: 0.9192 - val_loss: 0.3448 - val_categorical_accuracy: 0.9185
Epoch 149/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2437 - categorical_accuracy: 0.9184 - val_loss: 0.3852 - val_categorical_accuracy: 0.9165
Epoch 150/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2377 - categorical_accuracy: 0.9204 - val_loss: 0.3817 - val_categorical_accur

Epoch 193/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2326 - categorical_accuracy: 0.9244 - val_loss: 0.3792 - val_categorical_accuracy: 0.9165
Epoch 194/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2301 - categorical_accuracy: 0.9247 - val_loss: 0.3871 - val_categorical_accuracy: 0.9180
Epoch 195/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2339 - categorical_accuracy: 0.9236 - val_loss: 0.3722 - val_categorical_accuracy: 0.9165
Epoch 196/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2288 - categorical_accuracy: 0.9247 - val_loss: 0.3822 - val_categorical_accuracy: 0.9170
Epoch 197/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2342 - categorical_accuracy: 0.9247 - val_loss: 0.3479 - val_categorical_accuracy: 0.9178
Epoch 198/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2331 - categorical_accuracy: 0.9250 - val_loss: 0.3446 - val_cat

In [92]:
history = model.fit(np.array(shuffle_x_train),
                    np.array(shuffle_target),
                    epochs=200,
                   validation_data = (np.array([x_train_set[i:i+200] for i in range(104000,105000,50)]),
                     np.array([target_onehot[i:i+200] for i in range(104000,105000,50)])),
    batch_size=64)


Train on 1040 samples, validate on 20 samples
Epoch 1/200
1040/1040 [==============================] - 8s 8ms/step - loss: 0.9576 - categorical_accuracy: 0.8999 - val_loss: 0.6547 - val_categorical_accuracy: 0.8890
Epoch 2/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.5134 - categorical_accuracy: 0.9133 - val_loss: 0.4781 - val_categorical_accuracy: 0.8890
Epoch 3/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4070 - categorical_accuracy: 0.9133 - val_loss: 0.4451 - val_categorical_accuracy: 0.8890
Epoch 4/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.3784 - categorical_accuracy: 0.9134 - val_loss: 0.4275 - val_categorical_accuracy: 0.8890
Epoch 5/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.3577 - categorical_accuracy: 0.9134 - val_loss: 0.4119 - val_categorical_accuracy: 0.8890
Epoch 6/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.3431 - categorical_accuracy: 0

Epoch 49/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2412 - categorical_accuracy: 0.9153 - val_loss: 0.4214 - val_categorical_accuracy: 0.9120
Epoch 50/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2392 - categorical_accuracy: 0.9153 - val_loss: 0.3956 - val_categorical_accuracy: 0.9107
Epoch 51/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2353 - categorical_accuracy: 0.9154 - val_loss: 0.3981 - val_categorical_accuracy: 0.9120
Epoch 52/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2403 - categorical_accuracy: 0.9157 - val_loss: 0.4140 - val_categorical_accuracy: 0.9110
Epoch 53/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2363 - categorical_accuracy: 0.9156 - val_loss: 0.3651 - val_categorical_accuracy: 0.9133
Epoch 54/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2326 - categorical_accuracy: 0.9165 - val_loss: 0.4116 - val_categoric

1040/1040 [==============================] - 6s 6ms/step - loss: 0.2222 - categorical_accuracy: 0.9134 - val_loss: 0.4084 - val_categorical_accuracy: 0.8785
Epoch 98/200
1040/1040 [==============================] - 6s 5ms/step - loss: 0.2216 - categorical_accuracy: 0.9183 - val_loss: 0.5708 - val_categorical_accuracy: 0.7730
Epoch 99/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2202 - categorical_accuracy: 0.9162 - val_loss: 0.4328 - val_categorical_accuracy: 0.8955
Epoch 100/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2153 - categorical_accuracy: 0.9212 - val_loss: 0.4024 - val_categorical_accuracy: 0.9125
Epoch 101/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2193 - categorical_accuracy: 0.9202 - val_loss: 0.3738 - val_categorical_accuracy: 0.9118
Epoch 102/200
1040/1040 [==============================] - 5s 4ms/step - loss: 0.2210 - categorical_accuracy: 0.9209 - val_loss: 0.4081 - val_categorical_accurac

1040/1040 [==============================] - 4s 4ms/step - loss: 0.2147 - categorical_accuracy: 0.9221 - val_loss: 0.3149 - val_categorical_accuracy: 0.9128
Epoch 146/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2176 - categorical_accuracy: 0.9206 - val_loss: 0.3672 - val_categorical_accuracy: 0.9140
Epoch 147/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2110 - categorical_accuracy: 0.9239 - val_loss: 0.4686 - val_categorical_accuracy: 0.8733
Epoch 148/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2135 - categorical_accuracy: 0.9216 - val_loss: 0.3211 - val_categorical_accuracy: 0.9168
Epoch 149/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2141 - categorical_accuracy: 0.9220 - val_loss: 0.3801 - val_categorical_accuracy: 0.9145
Epoch 150/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2109 - categorical_accuracy: 0.9229 - val_loss: 0.4255 - val_categorical_accur

Epoch 193/200
1040/1040 [==============================] - 5s 4ms/step - loss: 0.2108 - categorical_accuracy: 0.9233 - val_loss: 0.4171 - val_categorical_accuracy: 0.9165
Epoch 194/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2103 - categorical_accuracy: 0.9247 - val_loss: 0.4854 - val_categorical_accuracy: 0.8727
Epoch 195/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2143 - categorical_accuracy: 0.9244 - val_loss: 0.4015 - val_categorical_accuracy: 0.9162
Epoch 196/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2112 - categorical_accuracy: 0.9227 - val_loss: 0.4413 - val_categorical_accuracy: 0.9162
Epoch 197/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2088 - categorical_accuracy: 0.9237 - val_loss: 0.3747 - val_categorical_accuracy: 0.9168
Epoch 198/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2079 - categorical_accuracy: 0.9235 - val_loss: 0.4002 - val_cat

In [93]:
results = model.predict(np.array(x_test_set[:34400]).reshape(172,200,3))
results = results.reshape(34400,3)

In [74]:
len(x_test_set)

27589

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test_set[:14150], result_class, labels=[0,1], average=None)

### Post-processing Section

In [94]:
result_class = []
for i in results:
#     if(max(i) < 0.5):
#         result_class.append(0)
#     else:
    if(np.argmax(i)==0):
        result_class.append(0)
    elif(np.argmax(i)==1):
        result_class.append(1)
    elif(np.argmax(i)==2):
        result_class.append(2)

In [122]:
"""
Creating list of events and 
removing events with duration lower than 50

Event tuple: [type, init-point, end-point, duration]


"""

brks = 0
accs = 0
num_events = 0
events = []
for i in range(len(result_class)):
    if(result_class[i]==1):
        brks += 1
    elif(result_class[i]==2):
        accs += 1
    else:
        if(brks > 50):
            events.append(("brake", i-brks, i-1, brks))
            print(num_events, "- braking event," ,"duration:", brks, " tsteps", "init:", i-brks ,"end:", i-1)
            brks = 0
            num_events += 1
                  
        elif(accs > 50):
            events.append(("acc", i-accs, i-1, accs))
            print(num_events, "- acceleration event," ,"duration:", accs, " tsteps", "init:", i-accs ,"end:", i-1)
            accs = 0
            num_events += 1

0 - acceleration event, duration: 125  tsteps init: 8075 end: 8199
1 - acceleration event, duration: 111  tsteps init: 11471 end: 11581
2 - acceleration event, duration: 144  tsteps init: 12056 end: 12199
3 - acceleration event, duration: 190  tsteps init: 23210 end: 23399
4 - braking event, duration: 55  tsteps init: 23690 end: 23744
5 - braking event, duration: 207  tsteps init: 26593 end: 26799
6 - braking event, duration: 68  tsteps init: 29629 end: 29696
7 - braking event, duration: 76  tsteps init: 30324 end: 30399
8 - acceleration event, duration: 164  tsteps init: 30636 end: 30799
9 - braking event, duration: 188  tsteps init: 31005 end: 31192
10 - acceleration event, duration: 99  tsteps init: 31501 end: 31599
11 - braking event, duration: 82  tsteps init: 32647 end: 32728


[('acc', 8075, 8199, 125),
 ('acc', 11471, 11581, 111),
 ('acc', 12056, 12199, 144),
 ('acc', 23210, 23399, 190),
 ('brake', 23690, 23744, 55),
 ('brake', 26593, 26799, 207),
 ('brake', 29629, 29696, 68),
 ('brake', 30324, 30399, 76),
 ('acc', 30636, 30799, 164),
 ('brake', 31005, 31192, 188),
 ('acc', 31501, 31599, 99),
 ('brake', 32647, 32728, 82)]

### Graph the results

In [96]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,34400)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,34400)),
    y = y_test_set,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Brake_Acc Model Classification')


py.offline.plot(fig, filename='my_data_brake_acc_onehot_200_softmax_categorical_crossentropy')


C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `my_data_brake_acc_onehot_200_softmax_categorical_crossentropy` didn't end with .html. Adding .html to the end of your file.



'my_data_brake_acc_onehot_200_softmax_categorical_crossentropy.html'

0.0

In [61]:

# test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')
test = pandas.read_csv('./data/Learning/MyData/version1/Test_Set.csv')


fig =tools.make_subplots(rows=1,cols=1)


test =  go.Scatter(
    x = list(range(0,27100)),
    y = test['x'],
    mode = 'lines',
    name='test'
)

fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Test Data')


py.offline.plot(fig, filename='brake_acc_test_data')


C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `brake_acc_test_data` didn't end with .html. Adding .html to the end of your file.



'brake_acc_test_data.html'

In [62]:
fig =tools.make_subplots(rows=2,cols=1)

loss_result =  go.Scatter(
    x = list(range(0,500)),
    y = history.history['loss'],
    mode = 'lines'
)

acc_result =  go.Scatter(
    x = list(range(0,500)),
    y = history.history['categorical_accuracy'],
    mode = 'lines'
)

fig.append_trace(loss_result, 1, 1)
fig.append_trace(acc_result, 2, 1)

fig['layout'].update(height=650, title = 'Loss, Accuracy')
py.offline.plot(fig, filename='multiple-axes-double')

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `multiple-axes-double` didn't end with .html. Adding .html to the end of your file.



'multiple-axes-double.html'

### Saving model

In [97]:
from keras.models import model_from_json
import os

# serialize model to JSON
model_json = model.to_json()
with open("./Learning_model/version1/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Learning_model/version1/model.h5")
print("Saved model to disk")

Saved model to disk


### Loading Model

In [48]:
from keras.models import model_from_json
import os

# load json and create model
json_file = open('./Learning_model/version1/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Learning_model/version1/model.h5")
print("Loaded model from disk")
model = loaded_model

Loaded model from disk


### Sensor Log Generated Data

In [106]:
my_data_part3 = pandas.read_csv('./MyData/22.4.2020.part3.csv')
my_data_part2 = pandas.read_csv('./MyData/22.4.2020.part2.csv')
my_data_round1 = pandas.read_csv('./MyData/22.4.2020/round1.csv')
my_data_round2 = pandas.read_csv('./MyData/22.4.2020/round2.csv')
my_data_round3 = pandas.read_csv('./MyData/22.4.2020/round3.csv')


# working_set = my_data[['accelerometerAccelerationX(G)','accelerometerAccelerationY(G)','accelerometerAccelerationZ(G)']][3200:3700]        

set_user_part3 = my_data_part3[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_part2 = my_data_part2[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round1 = my_data_round1[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round2 = my_data_round2[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round3 = my_data_round3[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]

######

my_data_brake_round4 = pandas.read_csv('./MyData/24.4.2020/brake_round4.csv')
my_data_brake_round5 = pandas.read_csv('./MyData/24.4.2020/brake_round5.csv')

set_user_brake_round4 = my_data_brake_round4[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_brake_round5 = my_data_brake_round5[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]

In [107]:
def RenderData(data, fig, init, end):
    acx =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][0] for i in range(init, end)],
        mode = 'lines',
        name = 'x'
    )
    

    acy =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][1] for i in range(init, end)],
        mode = 'lines',
        name='y'
    )

    acz =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][2] for i in range(init, end)],
        mode = 'lines',
        name='z'
    )
    fig.append_trace(acx,1,1)
    fig.append_trace(acy,1,1)
    fig.append_trace(acz,1,1)
    
    return fig

In [108]:
set_user_part2 = np.array(set_user_part2)
set_user_part2 = SmoothData(set_user_part2)
set_user_part3 = np.array(set_user_part3)
set_user_part3 = SmoothData(set_user_part3)
######
set_user_round1 = np.array(set_user_round1)
set_user_round2 = np.array(set_user_round2)
set_user_round3 = np.array(set_user_round3)
set_user_round1 = SmoothData(set_user_round1)
set_user_round2 = SmoothData(set_user_round2)
set_user_round3 = SmoothData(set_user_round3)

set_user_brake_round4 = np.array(set_user_brake_round4)
set_user_brake_round5 = np.array(set_user_brake_round5)
set_user_brake_round4 = SmoothData(set_user_brake_round4)
set_user_brake_round5 = SmoothData(set_user_brake_round5)

In [162]:
fig =tools.make_subplots(rows=1,cols=1)
RenderData(set_user_round1, fig,  8093, 8395)
fig['layout'].update(height=500, title = 'Test Data')
py.offline.plot(fig, filename='brake_acc_test_data')

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `brake_acc_test_data` didn't end with .html. Adding .html to the end of your file.



'brake_acc_test_data.html'

In [159]:
len(set_user_brake_round5)

47371